In [1]:
import os
import pandas as pd
import re
import zipfile
import tkinter as tk
from tkinter import filedialog, messagebox

In [2]:
def load_excel_data(file_path):
    excel_data = pd.read_excel(file_path)
    return excel_data[['Posting Date', 'Amount', 'Description']]

In [19]:
load_excel_data("C:/Users/Bingyan/Desktop/ML-file-process/test/test/CreditCardAsof05312024.xlsx")

,Posting Date,Amount,Description
0,2024-05-01,292.91,DD DOORDASH EARLYBIRD
1,2024-05-02,263.25,DD DOORDASH DOUGHZONE
2,2024-05-02,463.61,"4IMPRINT, INC"
3,2024-05-02,13.00,FEDEX607207744
4,2024-05-02,55.74,NEXTIVA*VOIP SERVICE
5,2024-05-03,229.83,DD DOORDASH NOBURURAM
6,2024-05-03,1650.00,UR CONFERENCE & EVENTS
7,2024-05-03,119.65,FEDEX607288797
8,2024-05-03,330.64,DNH*GODADDY.COM
9,2024-05-06,281.28,DD DOORDASH EARLYBIRD


In [16]:
def rename_files(directory, excel_data):
    for root, _, files in os.walk(directory):
        for file in files:
            match = re.match(r'(\d{8})_\$(\d+(\.\d{2})?)\.(pdf|jpg)', file)
            if match:
                date, amount, _, ext = match.groups()
                amount = float(amount.replace(',', ''))  # Convert amount to float
                
                entry = excel_data[(excel_data['Posting Date'] == date) & 
                                   (excel_data['Amount'] == amount)]
                
                if not entry.empty:
                    merchant = entry['Description'].values[0]
                    merchant_clean = re.sub(r'[^\w\s]', '', merchant).replace(' ', '_')
                    new_name = f"{date}_{merchant_clean}_${amount:.2f}.{ext}"
                    os.rename(os.path.join(root, file), os.path.join(root, new_name))
                    print(f"Renamed {file} to {new_name}")


In [17]:
rename_files("test/test", data)

In [18]:
import unittest
from unittest.mock import patch, mock_open, MagicMock
import pandas as pd
import os
from io import StringIO

# Assuming the functions are in a module named file_renaming
from file_renaming import load_excel_data, rename_files

class TestFileRenaming(unittest.TestCase):

    @patch('pandas.read_excel')
    def test_load_excel_data(self, mock_read_excel):
        # Mock data
        mock_data = pd.DataFrame({
            'Posting Date': ['05/03/2024', '05/06/2024'],
            'Amount': [1.65, 231.12],
            'Description': ['Eventbrite', 'Frying Fish Restaurant']
        })
        mock_read_excel.return_value = mock_data

        # Expected output
        expected_data = pd.DataFrame({
            'Posting Date': ['05032024', '05062024'],
            'Amount': [1.65, 231.12],
            'Description': ['Eventbrite', 'Frying Fish Restaurant']
        })

        result = load_excel_data('dummy_path')
        pd.testing.assert_frame_equal(result, expected_data)

    @patch('os.rename')
    @patch('os.walk')
    def test_rename_files(self, mock_os_walk, mock_os_rename):
        mock_os_walk.return_value = [
            ('/dummy_dir', [], ['05032024_$1.65.pdf', '05062024_$231.jpg'])
        ]

        excel_data = pd.DataFrame({
            'Posting Date': ['05032024', '05062024'],
            'Amount': [1.65, 231.12],
            'Description': ['Eventbrite', 'Frying Fish Restaurant']
        })

        rename_files('/dummy_dir', excel_data)

        mock_os_rename.assert_any_call(
            '/dummy_dir/05032024_$1.65.pdf', '/dummy_dir/05032024_Eventbrite_$1.65.pdf')
        mock_os_rename.assert_any_call(
            '/dummy_dir/05062024_$231.jpg', '/dummy_dir/05062024_Frying_Fish_Restaurant_$231.12.jpg')

if __name__ == '__main__':
    unittest.main()

ModuleNotFoundError: No module named 'file_renaming'